In [1]:
pip install git+ssh://git@github.com/dorianbrown/rank_bm25.git

  Cloning ssh://****@github.com/dorianbrown/rank_bm25.git to c:\users\josipk~1\appdata\local\temp\pip-req-build-apc088lp
  Created wheel for rank-bm25: filename=rank_bm25-0.2.1-py3-none-any.whl size=8574 sha256=fa33f0f5434a53cd48b3bf79b83d58bcba248ebd924ac086d06b6a35fc8eb5c9
  Stored in directory: C:\Users\JOSIPK~1\AppData\Local\Temp\pip-ephem-wheel-cache-ura0sc4r\wheels\b5\04\7a\a7a2d5c6113841f0d3bd39f9671cc7479f62bd33e2b63464d0
Successfully built rank-bm25
Note: you may need to restart the kernel to use updated packages.


In [2]:
from rank_bm25 import BM25Okapi
import glob
import sys
import os
import os.path 
import numpy as np
import string
from sklearn import preprocessing

In [20]:
## Importing bad and good bias words
################################### CHANGE ROUTE FOR CODE TO WORK ####################################################
os.chdir(r'C:\Users\JosipKoprcina\Desktop\IRProjectMovieScriptGenderBiasRetrieval')
bad_bias_words = open("BadBias.txt").read().split("\n")
good_bias_words = open("GoodBias.txt").read().split("\n")

## Initialzing needed dictionaries for single and paired words
bad_bias_unigrams = {}
bad_bias_bigrams = {}
good_bias_unigrams = {}
good_bias_bigrams = {}

## Adding bad bias words from basic text to dictionaries
for x in bad_bias_words:
    if(len(x.strip().split(" ")) == 2):
        bad_bias_bigrams[x.strip()] = 0
    else:
        bad_bias_unigrams[x.strip()] = 0
        
## Adding good bias words from basic text to dictionaries
for x in good_bias_words:
    if(len(x.strip().split(" ")) == 2):
        good_bias_bigrams[x.strip()] = 0
    else:
        good_bias_unigrams[x.strip()] = 0
        
## Importing movie scripts
################################### CHANGE ROUTE FOR CODE TO WORK ####################################################
os.chdir(r'C:\Users\JosipKoprcina\Desktop\IRProjectMovieScriptGenderBiasRetrieval\data')
myFiles = glob.glob('*.txt')
scripts = []
for file in myFiles:
    file = open(file,mode='r', encoding='utf-8-sig')
    scripts.append(file.read())
    file.close()
    
print(len(bad_bias_unigrams)+len(bad_bias_bigrams))
print(len(good_bias_unigrams)+len(good_bias_bigrams))

77
122


In [4]:
## Text Preprocessing

## Removing unneded white spaces and splitting scipts into tokens
scripts = [" ".join(doc.split()) for doc in scripts]
scripts = [doc.lower() for doc in scripts]
scripts = [doc.translate(str.maketrans("", "", string.punctuation)) for doc in scripts]
tokenized_corpus = [doc.split(" ") for doc in scripts]

## Creating pairs of tokens
tokenized_corpus_bigrams = []
for file in tokenized_corpus:
    current = []
    for i in range(len(file) - 1):
        current.append(file[i] + " " + file[i+1])
    tokenized_corpus_bigrams.append(current)

In [5]:
## Counting good and bad bias for each script and creating a weight list for all scripts
## Weight consists of number of different good/bad biases and the amount of times they occur (only second implemented for now)
bias_list = []

for i in range(len(scripts)):
    ## Increment dictionaries
    for word in tokenized_corpus[i]:
        if word in bad_bias_unigrams:
            if(bad_bias_unigrams[word] > 100):
                bad_bias_unigrams[word] += 0.01
            elif(bad_bias_unigrams[word] > 50):
                bad_bias_unigrams[word] += 0.1
            else:
                bad_bias_unigrams[word] += 1
        elif word in good_bias_unigrams:
            if(good_bias_unigrams[word] > 100):
                good_bias_unigrams[word] += 0.01
            elif(good_bias_unigrams[word] > 50):
                good_bias_unigrams[word] += 0.1
            else:
                good_bias_unigrams[word] += 1
    for word in tokenized_corpus_bigrams[i]:
        if word in bad_bias_bigrams:
            if(bad_bias_bigrams[word] > 100):
                bad_bias_bigrams[word] += 0.01
            elif(bad_bias_bigrams[word] > 50):
                bad_bias_bigrams[word] += 0.1
            else:
                bad_bias_bigrams[word] += 1
        elif word in good_bias_bigrams:
            if(good_bias_bigrams[word] > 100):
                good_bias_bigrams[word] += 0.01
            elif(good_bias_bigrams[word] > 50):
                good_bias_bigrams[word] += 0.1
            else:
                good_bias_bigrams[word] += 1
    
    ## Count bias and add to list of biases
    bad_bias = sum(bad_bias_unigrams.values()) + sum(bad_bias_bigrams.values())
    good_bias = sum(good_bias_unigrams.values()) + sum(good_bias_bigrams.values())
    bias =  good_bias-bad_bias*2
    bias_list.append(bias)
    
    ## Reset dictionaries to zero        
    bad_bias_unigrams = bad_bias_unigrams.fromkeys(bad_bias_unigrams, 0)
    bad_bias_bigrams = bad_bias_bigrams.fromkeys(bad_bias_bigrams, 0)
    good_bias_unigrams = good_bias_unigrams.fromkeys(good_bias_unigrams, 0)
    good_bias_bigrams = good_bias_bigrams.fromkeys(good_bias_bigrams, 0)

## Print bias_list for better visualization
bm25 = BM25Okapi(tokenized_corpus)

In [13]:
## Printing movie scripts ranked from most to least biased
bias_list_standardised = preprocessing.scale(bias_list)
bias = {}
for i in range(len(myFiles)):
    bias[myFiles[i]] = bias_list_standardised[i]

In [14]:
for w in sorted(bias, key=bias.get, reverse=False):
    print(w, bias[w])

Script_Master, The.txt -6.605960498622373
Script_Megamind.txt -5.128041676232752
Script_Thunderbirds.txt -4.6117381574941305
Script_Distinguished Gentleman, The.txt -4.424578131951335
Script_Shakespeare in Love.txt -4.295502252266652
Script_Producer, The.txt -4.269687076329717
Script_Sleepy Hollow.txt -3.998627728991888
Script_Moonrise Kingdom.txt -3.972812553054935
Script_12 Years a Slave.txt -3.805013909464863
Script_Pride and Prejudice.txt -3.7921063214963935
Script_Barry Lyndon.txt -3.6823918237644113
Script_Titanic.txt -3.553315944079729
Script_White Ribbon, The.txt -3.391971094473878
Script_Panic Room.txt -3.340340742600006
Script_Ali.txt -3.3016179786946034
Script_Jacket, The.txt -3.2951641847103663
Script_Perfect Creature.txt -3.230626244868023
Script_Star Wars_ Revenge of the Sith.txt -3.198357274946855
Script_Lord of the Rings_ Fellowship of the Ring, The.txt -3.1015503651833405
Script_Kung Fu Panda.txt -3.030558631356766
Script_Rush Hour 2.txt -2.8756675757351466
Script_Cinc

Script_ParaNorman.txt 0.7513646434044288
Script_Lost Highway.txt 0.7545915403965457
Script_A Serious Man.txt 0.770726025357131
Script_Arcade.txt 0.770726025357131
Script_Awakenings.txt 0.770726025357131
Script_Batman 2.txt 0.770726025357131
Script_Beavis and Butt-head Do America.txt 0.770726025357131
Script_Bling Ring, The.txt 0.770726025357131
Script_Hancock.txt 0.770726025357131
Script_Jaws 2.txt 0.770726025357131
Script_Marley & Me.txt 0.770726025357131
Script_Max Payne.txt 0.770726025357131
Script_Memento.txt 0.770726025357131
Script_Out of Sight.txt 0.770726025357131
Script_Roommate, The.txt 0.770726025357131
Script_Rushmore.txt 0.770726025357131
Script_South Park.txt 0.770726025357131
Script_Tall in the Saddle.txt 0.770726025357131
Script_American Graffiti.txt 0.8029949952783015
Script_Day the Earth Stood Still, The.txt 0.8029949952783015
Script_Last Flight, The.txt 0.8029949952783015
Script_Life of David Gale, The.txt 0.8029949952783015
Script_Moonstruck.txt 0.8029949952783015
S

In [15]:
for w in sorted(bias, key=bias.get, reverse=True):
    print(w, bias[w])

Script_Hudsucker Proxy, The.txt 4.155740970087929
Script_Schindler's List.txt 2.8036711303908786
Script_Kafka.txt 2.771402160469708
Script_Island, The.txt 2.6423262807850256
Script_Hebrew Hammer, The.txt 2.5745614439505684
Script_Invictus.txt 2.4164434913368313
Script_Little Athens.txt 2.4035359033683643
Script_Avatar.txt 2.287367611652149
Script_Thirteen Days.txt 2.190560701888637
Script_Girl with the Dragon Tattoo, The.txt 2.0937537921251255
Script_Minority Report.txt 2.0937537921251255
Script_Monte Carlo.txt 2.0937537921251255
Script_Authors Anonymous.txt 2.061484822203955
Script_Debt, The.txt 2.061484822203955
Script_King of Comedy, The.txt 2.029215852282784
Script_Bachelor Party.txt 1.9969468823616134
Script_L.A. Confidential.txt 1.9969468823616134
Script_Sister Act.txt 1.954997221464091
Script_At First Sight.txt 1.9324089425192723
Script_Big Blue, The.txt 1.9324089425192723
Script_Mud.txt 1.9098206635744532
Script_Mechanic, The.txt 1.9033668695902195
Script_All About Steve.txt 1.

Script_Willow.txt -0.5523017414108634
Script_American Madness.txt -0.5845707113320341
Script_Beloved.txt -0.5845707113320341
Script_Black Rain.txt -0.5845707113320341
Script_Croods, The.txt -0.5845707113320341
Script_Crying Game.txt -0.5845707113320341
Script_Diner.txt -0.5845707113320341
Script_Good Girl, The.txt -0.5845707113320341
Script_Hackers.txt -0.5845707113320341
Script_Incredibles, The.txt -0.5845707113320341
Script_Mission Impossible II.txt -0.5845707113320341
Script_Next.txt -0.5845707113320341
Script_Peggy Sue Got Married.txt -0.5845707113320341
Script_Precious.txt -0.5845707113320341
Script_RocknRolla.txt -0.5845707113320341
Script_Win Win.txt -0.5845707113320341
Script_American Splendor.txt -0.6168396812532047
Script_Ghostbusters.txt -0.6168396812532047
Script_His Girl Friday.txt -0.6168396812532047
Script_Notting Hill.txt -0.6168396812532047
Script_Trainspotting.txt -0.6168396812532047
Script_Walking Tall.txt -0.6168396812532047
Script_Zerophilia.txt -0.6168396812532047

In [16]:
## Importing queries
################################### CHANGE ROUTE FOR CODE TO WORK ####################################################
os.chdir(r'C:\Users\JosipKoprcina\Desktop\IRProjectMovieScriptGenderBiasRetrieval')
queries = open("Queries.txt").read().split("\n")

tokenized_queries = []
for query in queries:
    tokenized_queries.append(query.split(" "))

In [17]:
doc_scores = bm25.get_scores(tokenized_queries[4])
doc_scores_bias = [doc_scores[i] + bias_list[i] for i in range(len(doc_scores))]

movies = {}
movies_bias = {}
for i in range(len(myFiles)):
    movies[myFiles[i]] = doc_scores[i]
    movies_bias[myFiles[i]] = doc_scores_bias[i]

In [18]:
## Printing movie order of relevance to query before bias was added
for w in sorted(movies, key=movies.get, reverse=True):
    print(w, movies[w])

Script_Day the Earth Stood Still, The.txt 21.874880521596292
Script_Time Machine, The.txt 21.296580277001517
Script_Independence Day.txt 21.270326343562193
Script_Argo.txt 21.23931375625475
Script_Avengers, The (2012).txt 20.97362681114567
Script_Stuntman, The.txt 20.95943410595102
Script_Flash Gordon.txt 20.950139108880528
Script_Fifth Element, The.txt 20.94060234682184
Script_Mission to Mars.txt 20.925046320342123
Script_Arcade.txt 20.89845632110084
Script_After School Special.txt 20.890654712298023
Script_Doors, The.txt 20.88394027092391
Script_Soldier.txt 20.882944326726296
Script_Three Kings (Spoils of War).txt 20.801449268448728
Script_Broken Embraces.txt 20.783569364357156
Script_Serenity.txt 20.697397851436527
Script_Cinema Paradiso.txt 20.68877699485026
Script_Land of the Dead.txt 20.641048270866843
Script_Lost in Translation.txt 20.610017334749884
Script_Battle_ Los Angeles.txt 20.608358120578774
Script_Ed Wood.txt 20.58898582072075
Script_Star Trek_ First Contact.txt 20.5130

Script_Frozen.txt 15.336967212627297
Script_Saw.txt 15.32664642943656
Script_Inventing the Abbotts.txt 15.319140586944261
Script_9.txt 15.313821876637228
Script_Ruins, The.txt 15.311032465575785
Script_Misery.txt 15.306212121530809
Script_Confidence.txt 15.297374864193896
Script_Gone in 60 Seconds.txt 15.286575452140061
Script_Zerophilia.txt 15.272607718476308
Script_Witness.txt 15.228137646185898
Script_Croods, The.txt 15.212075598466164
Script_Rise of the Guardians.txt 15.166461917753452
Script_Being There.txt 15.155899466811313
Script_Heist.txt 15.13848047794603
Script_Tamara Drewe.txt 15.115984599275427
Script_Rambling Rose.txt 15.098186133123757
Script_Manhattan Murder Mystery.txt 15.071791677035801
Script_Nightmare on Elm Street, A.txt 15.064179570426656
Script_Interview with the Vampire.txt 15.06244480780494
Script_No Strings Attached.txt 15.036094295030301
Script_There's Something About Mary.txt 14.937981986112707
Script_Superbad.txt 14.927164147504133
Script_Kids.txt 14.907038

In [19]:
## Printing movie order of relevance to query after bias was added
for w in sorted(movies_bias, key=movies_bias.get, reverse=True):
    print(w, movies_bias[w])

Script_Hudsucker Proxy, The.txt 173.2917705691876
Script_Schindler's List.txt 132.46297274293332
Script_Kafka.txt 131.78658403856014
Script_Island, The.txt 126.41220966078619
Script_Hebrew Hammer, The.txt 124.71628953287347
Script_Invictus.txt 120.66825636488485
Script_Avatar.txt 119.19528512296526
Script_Little Athens.txt 115.74758254444185
Script_Thirteen Days.txt 114.29546635048287
Script_Girl with the Dragon Tattoo, The.txt 111.77949953070127
Script_Debt, The.txt 110.38147577574411
Script_Minority Report.txt 110.03274642572657
Script_Monte Carlo.txt 109.09390191127385
Script_Authors Anonymous.txt 108.31870219433381
Script_Mechanic, The.txt 107.58890325622663
Script_King of Comedy, The.txt 107.18759430561994
Script_Traffic.txt 106.51204533687063
Script_L.A. Confidential.txt 106.24588918308288
Script_Bachelor Party.txt 106.16439397496822
Script_At First Sight.txt 105.57911656029736
Script_Mud.txt 105.10930571430451
Script_Sister Act.txt 105.01309695145596
Script_Simone.txt 104.670064

Script_Highlander_ Endgame.txt 27.53241752597513
Script_Pirates of the Caribbean.txt 27.38803319101715
Script_Tamara Drewe.txt 27.11598459927543
Script_Social Network, The.txt 27.072557974236727
Script_Precious.txt 26.967564221992852
Script_Martha Marcy May Marlene.txt 26.91907309331314
Script_Dumb and Dumber.txt 26.895909229693437
Script_Phone Booth.txt 26.882285260088544
Script_Peggy Sue Got Married.txt 26.863507164875763
Script_RocknRolla.txt 26.802207137290416
Script_Win Win.txt 26.75545925592256
Script_Observe and Report.txt 26.733411810134424
Script_Pirates of the Caribbean_ Dead Man's Chest.txt 26.698561817187183
Script_Beloved.txt 26.67784858044269
Script_Willow.txt 26.661447682027696
Script_Chasing Amy.txt 26.584465898318037
Script_Lake Placid.txt 26.560122845112794
Script_Black Rain.txt 26.28783443448484
Script_Reservoir Dogs.txt 26.09805494718626
Script_Ghostbusters.txt 25.89332804958347
Script_Drive Angry.txt 25.765663827683102
Script_His Girl Friday.txt 25.68116945911512
S

Script_Up.txt -19.789950374948702
Script_Yes Man.txt -21.83338219790794
Script_When a Stranger Calls.txt -23.32672687620311
Script_Being Human.txt -24.002134202817153
Script_Mrs. Brown.txt -25.365072932049003
Script_Airplane.txt -25.381488819449107
Script_Leaving Las Vegas.txt -25.97357961412157
Script_Die Hard 2.txt -26.195928713250293
Script_Crouching Tiger, Hidden Dragon.txt -27.72357555839874
Script_Elizabeth_ The Golden Age.txt -33.26858602628936
Script_Star Wars_ Attack of the Clones.txt -35.11745275391402
Script_Rear Window.txt -36.40848822177272
Script_Other Boleyn Girl, The.txt -36.418215477799464
Script_Les Miserables.txt -37.53763264961941
Script_Mute Witness.txt -38.957014874163384
Script_Cincinnati Kid, The.txt -40.30909795942347
Script_Rush Hour 2.txt -43.67712094040623
Script_Star Wars_ Revenge of the Sith.txt -51.76231776322818
Script_Lord of the Rings_ Fellowship of the Ring, The.txt -51.76499366908064
Script_Kung Fu Panda.txt -53.13696239388161
Script_Perfect Creature